# Somewhat Square Sudoku (2025 January puzzle Janestreet)

Statement: https://www.janestreet.com/puzzles/somewhat-square-sudoku-index/

Janestreet solution: https://www.janestreet.com/puzzles/somewhat-square-sudoku-solution/

# 0. The Solution Method

For each digit $r \in \{1,3,4,6,7,8,9\}$ (we cannot remove 0, 2, 5), let $D_r$ denote the largest possible GCD of the 9-digit numbers formed by the rows in a completed Sudoku puzzle that does not use the digit $r$.. Our goal is to find the maximum value among these,
$$
D := \max_{r \in \{1,3,4,6,7,8,9\}} D_r,
$$
and to identify a corresponding Sudoku puzzle that achieves this value.

We proceed by fixing a digit $r$. Since $D_r$ divides every row sum in such a Sudoku, it must also divide the total sum of all rows. This total sum is $111,111,111 \times (45 - r)$, because the sum of digits 0 through 9 is 45, and we are omitting $r$. In other words,
$$
D_r \mid 111,111,111 \times (45 - r).
$$
Therefore, $D_r$ must be a divisor of $111,111,111 \times (45 - r)$.

Our method is as follows:

1. For each $r\in \{1,3,4,6,7,8,9\}$, we enumerate all divisors of $111,111,111 \times (45 - r)$.
2. For each divisor, we check if it could be the GCD of a Sudoku row. A valid row would be an 8-digit number (since the first cell is 0) with all digits distinct, containing the digits 2 and 5 (which are always present), and excluding 0 as a leading digit and $r$ entirely. If a divisor cannot form such a number, we discard it as a candidate for $D_r$.
3. We store the remaining candidate divisors along with their associated $r$.
4. We then sort these (divisor, $r$) pairs in descending order by the divisor, omitting 1, as any completed Sudoku has a GCD of at least 1.

Finally, we attempt to solve for a Sudoku puzzle that does not use the digit $r$ and where every completed row is a multiple of the current candidate divisor. Since we process the candidates in descending order, the first valid Sudoku we find must have the maximum possible GCD, $D$. This is because any larger GCD would have been a divisor of $111,111,111 \times (45 - r)$ and would have been listed earlier.

In [6]:
import math

In [7]:
box_i = {(r, c): (r // 3) * 3 + (c // 3) for r in range(9) for c in range(9)}

# Pre-filled cells (initial given values)
given = {
    (0, 7): 2,
    (1, 8): 5,
    (2, 1): 2,
    (3, 2): 0,
    (5, 3): 2,
    (6, 4): 0,
    (7, 5): 2,
    (8, 6): 5
}

In [8]:
GREEN = "\033[92m"  # bright green
BLUE = "\033[94m"   # bright blue
RED  = "\033[91m"   # bright red
RESET = "\033[0m"

def printing_grid(grid, is_sol=False):
    for i, row in enumerate(grid):
        parts = []
        for j, val in enumerate(row):
            s = "." if val == -1 else str(val)

            #given (red) overrides row-blue
            if (i, j) in given:
                s = f"{RED}{s}{RESET}"
            elif i == 4 and is_sol:
                s = f"{BLUE}{s}{RESET}"

            parts.append(s)

            #vertical separator every 3 numbers, except at the end
            if (j + 1) % 3 == 0 and j < 8:
                parts.append("|")

        line = " ".join(parts)

        #horizontal separator every 3 rows, except at the end
        if i > 0 and i % 3 == 0:
            #Build horizontal line matching the 3x3 boxes
            horiz_line = "------+-------+------"
            print(horiz_line)

        print(line)

In [9]:
def remove_and_search(rem,gcd):

    # --------------------------
    # Setup
    # --------------------------
    print_every_N = 100_000
    total_sudoku = 0

    # Track used digits in rows, cols, blocks for O(1) checking
    row_used = {r:set() for r in range(9)}
    col_used = {c:set() for c in range(9)}
    box_used = {b:set() for b in range(9)}

    # Construct the grid
    grid = [[-1 for _ in range(9)] for _ in range(9)]

    for (r, c), v in given.items():
        # fill the grid
        grid[r][c] = v
        # fill used sets
        row_used[r].add(v)
        col_used[c].add(v)
        box_used[box_i[(r, c)]].add(v)

    # we are searching a sudoku without the digit 'rem'
    candidates = [x for x in range(10) if x!=rem]

    def row_number(row): #only called when a row is completed
        n = 0
        for v in row:
            n = n * 10 + v
        return n

    def solveSudoku(grid, r=0, c=0):
        # End of grid
        if r == 9:
            row_val = row_number(grid[-1])
            #we've reached this point only if all previous rows are multiples of gcd
            if row_val % gcd == 0: #check if it is also a multiple of gcd
                print("...")
                print(f"{GREEN}✅ SOLUTION FOUND!!{RESET} When removing {rem} with gcd={gcd}.")
                printing_grid(grid,is_sol=True)
                print("...")
                return True
            return False

        # End of row
        if c == 9:
            row_val = row_number(grid[r])
            if row_val % gcd != 0: #stop if current row is not a multiple of gcd
                return False
            return solveSudoku(grid, r + 1, 0)

        # Skip filled cell
        if grid[r][c] > -1:
            return solveSudoku(grid, r, c + 1)

        box = box_i[(r, c)]
        for num in candidates:
            if num not in row_used[r] and num not in col_used[c] and num not in box_used[box]:
                grid[r][c] = num
                row_used[r].add(num)
                col_used[c].add(num)
                box_used[box].add(num)

                if solveSudoku(grid, r, c + 1):
                    return True  #early stop when the solution is found

                grid[r][c] = -1
                row_used[r].remove(num)
                col_used[c].remove(num)
                box_used[box].remove(num)

        return False

    found = solveSudoku(grid)
    if not found:
        print(f"No solution found for (gcd,rem)=({gcd},{rem}).")
    return found


In [10]:
# A function to print all prime factors of
# a given number n
def primeFactors(n):
    sol = []
    # Count and store all factors of 2
    while n % 2 == 0:
        sol.append(2)
        n = n // 2

    # n is now odd - check only odd divisors from 3 onward
    for i in range(3,int(math.sqrt(n))+1,2):

        # While i divides n, store i and divide n
        while n % i== 0:
            sol.append(i)
            n = n // i

    # Condition if n is a prime
    # number greater than 2
    if n > 2:
        sol.append(n)
    return sol

def get_divisors(n):
    divisors = []
    if n == 1:
        divisors.append(1)
    elif n > 1:
        prime_factors = primeFactors(n)
        divisors = [1]
        last_prime = 0
        factor = 0
        slice_len = 0
        # Generate all divisors from prime factors
        for prime in prime_factors:
            if last_prime != prime:
                slice_len = len(divisors)
                factor = prime
            else:
                factor *= prime
            for i in range(slice_len):
                divisors.append(divisors[i] * factor)
            last_prime = prime
        divisors.sort()
    return divisors

In [12]:
S_tot = 45 #this is 0+1+2+...+9

all_possible_gcd = []

for rem in [1,3,4,6,7,8,9]:
    S = S_tot - rem
    #sorted in decreasing order
    #Skip divisor 1 since any valid sudoku row would have GCD at least 1
    divisors = get_divisors(111111111*S)[1:]

    for d in divisors:
        if d<=98765432:
            all_possible_gcd.append((d,rem))
        else:
            break

all_possible_gcd.sort(key=lambda x: x[0], reverse=True)

for gcd, rem in all_possible_gcd:
    k=1
    search_sol = False
    while k*gcd<=98765432 and not search_sol: # Ensure the number has at most 8 digits (since one row must start with 0)
        # Check if all the numbers are all different, and 2,5 are in the number
        # No '0' in s because we require an 8-digit number (leading zero)
        s = str(k*gcd)
        if len(s) == 8 and len(set(s)) == 8 and '2' in s and '5' in s and str(rem) not in s:
            search_sol = True
        k+=1
    if search_sol:
        print(f"... Searching for (gcd,rem)=({gcd},{rem})")
        if remove_and_search(rem,gcd):
            break


... Searching for (gcd,rem)=(49382716,1)
No solution found for (gcd,rem)=(49382716,1).
... Searching for (gcd,rem)=(24691358,1)
No solution found for (gcd,rem)=(24691358,1).
... Searching for (gcd,rem)=(24691358,7)
No solution found for (gcd,rem)=(24691358,7).
... Searching for (gcd,rem)=(12345679,1)
No solution found for (gcd,rem)=(12345679,1).
... Searching for (gcd,rem)=(12345679,4)
...
✅ SOLUTION FOUND!! When removing 4 with gcd=12345679.
3 9 5 | 0 6 1 | 7 2 8
0 6 1 | 7 2 8 | 3 9 5
7 2 8 | 3 9 5 | 0 6 1
------+-------+------
9 5 0 | 6 1 7 | 2 8 3
2 8 3 | 9 5 0 | 6 1 7
6 1 7 | 2 8 3 | 9 5 0
------+-------+------
8 3 9 | 5 0 6 | 1 7 2
5 0 6 | 1 7 2 | 8 3 9
1 7 2 | 8 3 9 | 5 0 6
...
